In [4]:
import numpy as np
import scipy.linalg
np.set_printoptions(linewidth = 150, precision = 4, suppress = True)

In [5]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(np.float64)
# Only take 10 columns for testing.
# np.random.seed(4)
def sample_hermitian(mat, choices):
    return mat[:, choices][choices, :]
# data = sample_hermitian(data, np.random.choice(data.shape[0], 10))
# Synthesize columns holding observations, reproducing the cov matrix.
raw_data = scipy.linalg.sqrtm(data)
np.testing.assert_allclose(data, raw_data.T @ raw_data)

In [6]:
nodes = (
    np.loadtxt('../NC-K7-Trace-Nodes.csv', delimiter=',', dtype=str)
)[1:].astype(np.float64).astype(int)
nodes[:, -1]

array([-1, -1, -1,  0, -1, -1, -1,  0, -1, -1, -1, -1,  1,  0, -1,  0, -1,  0, -1,  1,  0,  0, -1, -1, -1, -1,  0,  0,  0,  0,  0, -1,  0, -1, -1,
       -1, -1,  0, -1, -1,  0,  0, -1,  0,  0,  0,  0, -1, -1,  0,  0,  0, -1, -1, -1,  0, -1,  0,  0,  0, -1,  0,  0, -1, -1, -1, -1,  0,  0,  0,
       -1, -1, -1,  0, -1, -1, -1,  0, -1,  0,  0, -1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0])

In [1]:
import tensorflow as tf

In [24]:
@tf.function
def combination(n, p, x):
    result = tf.TensorArray(ncr.dtype, size=p)
    r = tf.constant(0, ncr.dtype)
    k = tf.constant(0, ncr.dtype)
    next_value = tf.constant(0, ncr.dtype)
    for i in tf.range(p-1):
        next_value += 1
        index = tf.cast(i, tf.int64)
        r = ncr[n-next_value, p-(index+1)]
        k += r
        while k < x:
            next_value += 1
            r = ncr[n-next_value, p-(index+1)]
            k += r
        k -= r
        result = result.write(i, next_value)
    result = result.write(p-1, next_value + x - k)
    return result.stack()
@tf.function
def combinations_batch(n, k, start, limit):
    return tf.map_fn(
        lambda i: combination(n, k, i),
        tf.range(start, limit),
        dtype=tf.int64)
@tf.function
def select_hermitian(arr, inds):
    # Select rows
    arr = tf.gather(arr, inds)
    # Select columns using transpose
    return tf.gather(tf.transpose(arr), inds)
@tf.function
def select_columns(arr, col_inds):
    return tf.transpose(
        tf.gather(tf.transpose(arr), col_inds))

In [21]:
np.random.seed(0)
selected_node = np.random.choice(nodes.shape[1])
selected_inds, = np.where(nodes[:, selected_node] == 1)
allowed_inds, = np.where(nodes[:, selected_node] == -1)
selected_inds, len(selected_inds), len(allowed_inds)

(array([20, 83, 85]), 3, 77)

In [18]:
import scipy.special

In [38]:
u = tf.linalg.svd(
    select_columns(raw_data, tf.constant(selected_inds))
)[1][:, 0]
# sigma (singular value which could be computed from svd)
sigma = tf.linalg.norm(
    tf.linalg.matmul(
        select_columns(raw_data, tf.constant(selected_inds)),
        u[:, None],
        adjoint_a=True))

In [39]:
# Spectral radius of the n x n, rank one matrix,
# column * u' * u * column' for each column of raw_data.
# We can take multiple columns of raw_data and take the
# Gram matrix, instead of just one. Summing up A_op_norm
# for the given columns is an argument that comes from
# the trace of the resulting matrix and the fact that it
# is rank-one.
A_op_norm = tf.square(
    tf.linalg.matmul(
        raw_data,
        u[:, None],
        adjoint_a=True))